<a href="https://colab.research.google.com/github/AdrianDiazGaxiola/Proyecto-Final/blob/main/IPCC_Procesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Este código se empleará para limpiar los datos de los archivos IPCC de los años 1990 a 2019

In [1]:
!pip install numpy pandas

Aquí se leen los archivos con los datos en bruto y se guardan como dataframes en un diccionario

In [16]:
import pandas as pd

IPCC = {}

for i in range(1990,2020):
  url = "https://github.com/AdrianDiazGaxiola/Proyecto-Final/raw/IPCC/IPCC_%d.csv" % i
  IPCC[i] = pd.read_csv(url, encoding = 'ISO-8859-1')

Se prueba que los archivos se leyeron correctamente, descargando uno de ellos

In [21]:
df = IPCC[1990]

df.to_csv("IPCC 90 Prueba.csv", encoding='ISO-8859-1', index=False)
from google.colab import files
files.download("IPCC 90 Prueba.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>